>### SENet

In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif']=['Arial Unicode MS'] 
plt.rcParams['axes.unicode_minus']=False 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

#### Model

In [25]:
class SEModule(nn.Module):
    def __init__(self, in_chans, reduction=16):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1) #Squeeze (1,1,c)
        self.fc1 = nn.Conv2d(in_chans, in_chans // reduction, kernel_size=1) #Excitation (1,1,c/r)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(in_chans // reduction, in_chans, kernel_size=1) #(1,1,c)
        self.sigmoid = nn.Sigmoid() #每个feature的权重

    def forward(self, x):
        inputs = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return inputs * x #加权features

In [26]:
class Bottleneck(nn.Module): #不同bottleneck共用的forward方法
    def forward(self, x):
        residual = x
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        
        if self.downsample is not None:
            residual = self.downsample(residual)
        x = self.se_module(x) + residual
        x = self.relu(x)
        return x
    
class SEBottleneck(Bottleneck): #继承forward方法
    expansion = 4
    def __init__(self, in_chans, out_chans, groups, reduction, stride=1, downsample=None):
        super(SEBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_chans, out_chans * 2, kernel_size=1, bias=False) #(h,w,c)
        self.bn1 = nn.BatchNorm2d(out_chans * 2)
        self.conv2 = nn.Conv2d(out_chans * 2, out_chans * 4, kernel_size=3,
                               stride=stride, padding=1, groups=groups, bias=False)
        self.bn2 = nn.BatchNorm2d(out_chans * 4)
        self.conv3 = nn.Conv2d(out_chans * 4, out_chans * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_chans * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(out_chans * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride

class SEResNetBottleneck(Bottleneck): 
    expansion = 4
    def __init__(self, in_chans, out_chans, groups, reduction, stride=1, downsample=None):
        super(SEResNetBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_chans, out_chans, kernel_size=1, stride=stride, bias=False) #(h,w,c)
        self.bn1 = nn.BatchNorm2d(out_chans)
        self.conv2 = nn.Conv2d(out_chans, out_chans, kernel_size=3, padding=1, groups=groups, bias=False)
        self.bn2 = nn.BatchNorm2d(out_chans)
        self.conv3 = nn.Conv2d(out_chans, out_chans * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_chans * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(out_chans * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride

class SEResNeXtBottleneck(Bottleneck): 
    expansion = 4
    def __init__(self, in_chans, out_chans, groups, reduction, stride=1, 
                 downsample=None, base_width=4):
        super(SEResNeXtBottleneck, self).__init__()
        width = int(math.floor(out_chans * (base_width / 64)) * groups)
        self.conv1 = nn.Conv2d(in_chans, width, kernel_size=1, stride=1, bias=False) #(h,w,c)
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=stride, padding=1, groups=groups, bias=False)
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = nn.Conv2d(width, out_chans * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_chans * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(out_chans * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride

In [27]:
class SENet(nn.Module):
    def __init__(self, block, layers, groups, reduction=16, dropout_p=0.2,
                 in_chans=128, input_3x3=True, downsample_kernel_size=3,
                 downsample_padding=1, num_classes=1000):
        super(SENet, self).__init__()
        self.in_chans = in_chans
        if input_3x3:
            layer0_modules = [
                ('conv1', nn.Conv2d(3, 64, 3, stride=2, padding=1, bias=False)),
                ('bn1', nn.BatchNorm2d(64)),
                ('relu1', nn.ReLU(inplace=True)),
                ('conv2', nn.Conv2d(64, 64, 3, stride=1, padding=1, bias=False)),
                ('bn2', nn.BatchNorm2d(64)),
                ('relu2', nn.ReLU(inplace=True)),
                ('conv3', nn.Conv2d(64, in_chans, 3, stride=1, padding=1, bias=False)),
                ('bn3', nn.BatchNorm2d(in_chans)),
                ('relu3', nn.ReLU(inplace=True)),
            ]
        else:
            layer0_modules = [
                ('conv1', nn.Conv2d(3, in_chans, kernel_size=7, stride=2, padding=3, bias=False)),
                ('bn1', nn.BatchNorm2d(in_chans)),
                ('relu1', nn.ReLU(inplace=True)),
            ]

        layer0_modules.append(('pool', nn.MaxPool2d(3, stride=2,ceil_mode=True)))
        
        self.layer0 = nn.Sequential(OrderedDict(layer0_modules))
        self.layer1 = self._make_layer(block, 64, layers[0], groups)
        self.layer2 = self._make_layer(block, 128, layers[1], groups, 2, 
                                       downsample_kernel_size, downsample_padding)
        self.layer3 = self._make_layer(block, 256, layers[2], groups, 2, 
                                       downsample_kernel_size, downsample_padding)
        self.layer4 = self._make_layer(block, 512, layers[3], groups, 2, 
                                       downsample_kernel_size, downsample_padding)
        self.avg_pool = nn.AvgPool2d(7, stride=1)
        self.dropout = nn.Dropout(dropout_p) if dropout_p is not None else None
        self.last_linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_chans, n_blocks, groups, stride=1,
                    downsample_kernel_size=1, downsample_padding=0, reduction=16):
        downsample = None
        if stride != 1 or self.in_chans != out_chans * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_chans, out_chans * block.expansion,
                          kernel_size=downsample_kernel_size, stride=stride,
                          padding=downsample_padding, bias=False),
                nn.BatchNorm2d(out_chans * block.expansion),
            )

        layers = []
        layers.append(block(self.in_chans, out_chans, groups, reduction, stride, downsample))
        self.in_chans = out_chans * block.expansion
        for i in range(1, n_blocks):
            layers.append(block(self.in_chans, out_chans, groups, reduction))
        return nn.Sequential(*layers)
    
    def forward(self, x): #(224,224,3)
        x = self.layer0(x) #(56,56,64)
        x = self.layer1(x) #
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avg_pool(x)
        
        if self.dropout is not None:
            x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x
    

In [28]:
structure = {
 'SENet154': SENet(SEBottleneck, [2, 2, 2, 2], groups=64, reduction=16, dropout_p=0.2,
                 in_chans=128, input_3x3=True, downsample_kernel_size=3,
                 downsample_padding=1, num_classes=1000),
 'SE-ResNet': SENet(SEResNetBottleneck, [2, 2, 2, 2], groups=1, reduction=16, dropout_p=None,
                 in_chans=64, input_3x3=False, downsample_kernel_size=1,
                 downsample_padding=0, num_classes=1000),
 'SE-ResNeXt': SENet(SEResNeXtBottleneck, [2, 2, 2, 2], groups=32, reduction=16, dropout_p=None,
                 in_chans=64, input_3x3=False, downsample_kernel_size=1,
                 downsample_padding=0, num_classes=1000),
 'FaceBagNet_model_A': SENet(SEResNeXtBottleneck, [2, 2, 2, 2], groups=32, reduction=16, dropout_p=None, 
                  in_chans=64, input_3x3=False, downsample_kernel_size=1, downsample_padding=0,
                  num_classes=1000),
 'FaceBagNet_model_B': SENet(SEResNeXtBottleneck, [2, 4, 4, 2], groups=32, reduction=16, dropout_p=None, 
                  in_chans=64, input_3x3=False, downsample_kernel_size=1, downsample_padding=0,
                  num_classes=1000),
 'FaceBagNet_model_C': SENet(SEResNeXtBottleneck, [3, 4, 4, 3], groups=16, reduction=16, dropout_p=None, 
                  in_chans=64, input_3x3=False, downsample_kernel_size=1, downsample_padding=0,
                  num_classes=1000)
}

In [540]:
model = structure['SE-ResNeXt']
summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5          [-1, 128, 56, 56]           8,192
       BatchNorm2d-6          [-1, 128, 56, 56]             256
              ReLU-7          [-1, 128, 56, 56]               0
            Conv2d-8          [-1, 128, 56, 56]           4,608
       BatchNorm2d-9          [-1, 128, 56, 56]             256
             ReLU-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          32,768
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,